<a href="https://colab.research.google.com/github/WhiteHum/Application-security/blob/main/2_04_Probability_Bayes_and_Spam_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Probability, Bayes, and Spam/Phish

## Overview

In this lab, you will take two approaches to categorizing messages as Ham or Spam.  You will begin with a basic statistics approach and then move toward the application of probabilities through an implementation of a Bayesian inference approach toward message categorization.

## Goals

By the end of this lab, you should be able to:

 * Apply descriptive statistics to textual data in interesting ways
 * Apply probability theory to classification problems
 * Implement a Bayesian inference classifier
 
## Estimated Time: 60 minutes


# <img src="../images/task.png" width=20 height=20> Task 4.1

Please use the next cell to import both the Pandas and Numpy libraries.  Import Numpy as np and Pandas as pandas.

In [ ]:
import numpy as np
import pandas as pd

To accomplish this lab we need example emails to work with.  Included in the data directory we have email from Enron corporation that was made public following their collapse in the early 2000's.  For our convenience, we have done some minor preprocessing to place all spam emails into the *spam* path and the non-spam into the *ham* path.

For both approaches, we need a corpus, or body, of text to work with.  Our first tasks will involve loading this corpus.  At the same time, we will want to set aside some of our ham and spam so that we have examples to evaluate our approaches.

# <img src="../images/task.png" width=20 height=20> Task 4.2

We want to write Python code to load the ham and spam into two arrays, one of ham and the other of spam.  Let's start by implementing a function that we can use to retrieve a list of files.  The function should allow us to pass a starting path and return a list of all of the files found in that path.

In [ ]:
import os

def get_file_list(starting_directory="../Data/Enron"):
    final_list = list()
    files = os.listdir(starting_directory)
    for file in files:
        file_name = os.path.join(starting_directory, file)
        if os.path.isdir(file_name):
            final_list = final_list + get_file_list(file_name)
        else:
            final_list.append(file_name)
    return final_list

all_files = get_file_list(starting_directory="../Data/Enron")
ham_files = get_file_list(starting_directory="../Data/Enron/ham")
spam_files = get_file_list(starting_directory="../Data/Enron/spam")
print(f'There are {len(all_files)} files total, {len(ham_files)} ham and {len(spam_files)} spam.')
print(all_files[:5], all_files[-5:])


There are 31716 files total, 16902 ham and 14814 spam.
['../Data/Enron\\ham\\beck-s\\2001_plan\\1', '../Data/Enron\\ham\\beck-s\\2001_plan\\2', '../Data/Enron\\ham\\beck-s\\2001_plan\\3', '../Data/Enron\\ham\\beck-s\\2001_plan\\4', '../Data/Enron\\ham\\beck-s\\2001_plan\\5'] ['../Data/Enron\\spam\\SH\\SA\\20030228_spam_2\\01396.e80a10644810bc2ae3c1b58c5fd38dfa', '../Data/Enron\\spam\\SH\\SA\\20030228_spam_2\\01397.f75f0dd0dd923faefa3e9cc5ecb8c906', '../Data/Enron\\spam\\SH\\SA\\20030228_spam_2\\01398.8ca7045aae4184d56e8509dc5ad6d979', '../Data/Enron\\spam\\SH\\SA\\20030228_spam_2\\01399.2319643317e2c5193d574e40a71809c2', '../Data/Enron\\spam\\SH\\SA\\20030228_spam_2\\01400.b444b69845db2fa0a4693ca04e6ac5c5']


There appears to be a pretty decent split of data with a bit more than 50% ham and a bit less than 50% spam.  Next, we need to read in the words that make up each email.

As we read in the words, it makes sense that we only want to keep things that are *words,* not numbers, or other non-wordlike things that might be present.  We're not going to be exhaustive about this, but we should make a good-faith effort to clean the data a bit.  A regular expression makes sense for this task.

In Python, we can create a regular expression using the *re* package.  After importing it, we can `compile()` a regular expression that can then be used to help us to parse our data.  In this case, it makes sense to discard everything that is not a word or a space.  Why spaces?  Because the spaces serve as our word separators.  Consider this expression:

```
regex = re.compile("[^\w\s]")
```

This states that we want a regular expression that matches everything that is *not* (^) in the *set* ([]) that contains either *words*  or *whitespace*.

When loading this text, we should bear in mind that Python strings are case sensitive.  Let's force everything to lowercase as a standard.

There's one other small challenge.  ***We do not want the mail headers included in our solution.***

# <img src="../images/task.png" width=20 height=20> Task 4.3

Build a Python function that returns a list of all of the words from a specified file.  The function should take care to skip over the email headers.

In [ ]:
import re
    
def get_words(file_name):
    exclusions = [
        "mime-version:",
        "content-t",
        "x-",
    ]

    regex = re.compile("[^\w\s]")
    words = list()
    with open(file_name, encoding='utf8', errors='ignore') as f:    
        text = f.readlines()
        finished_header = False
        for line in text:
            line = line.lower()
            if finished_header:
                start = [line.startswith(i) for i in exclusions]
                if not True in start:
                    for word in re.sub(regex, '', line).split():
                        words.append(word)
        
            elif line[0:8] == "subject:":
            
                finished_header = True
    return words

get_words('../data/Enron/ham/lokay-m/enron_t_s/25')

['notice',
 'to',
 'all',
 'ets',
 'personnel',
 'on',
 'october',
 '11',
 '2000',
 'each',
 'of',
 'the',
 'enron',
 'pipelines',
 'will',
 'file',
 'for',
 'ferc',
 'approval',
 'of',
 'a',
 'new',
 'set',
 'of',
 'standards',
 'of',
 'conduct',
 'under',
 'the',
 'fercs',
 'marketing',
 'affiliate',
 'rule',
 'please',
 'go',
 'to',
 'wwwotsenroncom',
 'to',
 'review',
 'the',
 'new',
 'standards',
 'of',
 'conduct',
 'for',
 'the',
 'pipelines',
 'for',
 'which',
 'you',
 'work',
 'click',
 'on',
 'ets',
 'legal',
 'department',
 'then',
 'click',
 'on',
 'antitrust',
 'handbook',
 'standards',
 'of',
 'conduct',
 'then',
 'click',
 'on',
 'the',
 'relevant',
 'pipelines']

With these functions completed, we'd like to read in the words from the corpus and store them.  We could store them in a number of ways, but for efficiency's sake we will use *sets.*  A set is very similar to a list of unordered values except that it is a list of *unique* values.  In other words, we can add the same value to a set 1,000 times and there will still only be one copy of that value in the set.

# <img src="../images/task.png" width=20 height=20> Task 4.4

Let's create two sets.  The first will contain words from ham, the second words from spam messages.  Build these sets from the first 10,000 ham and the first 10,000 spam messages.

In [ ]:
ham_words = set()
spam_words = set()
for i in range(10000):
    _ = [ham_words.add(word) for word in get_words(ham_files[i])]
    _ = [spam_words.add(word) for word in get_words(spam_files[i])]
    
len(ham_words),len(spam_words)

(73062, 250700)

We now have two sets of words.  Let's try a naive statistics approach.  Let's take five messages from the remaining ham and spam messages that have *not* been loaded and work out how many of the words in each message appear in our ham or spam sets.

# <img src="../images/task.png" width=20 height=20> Task 4.5

Write a function that will load in an email message.  The function then returns a three element tuple that gives us the number of words in the email, the number of words that appear in the ham set, and the number of words that appear in the spam set.  After writing this function, process the next 4000 each of the ham and spam files (files 10000 through 14000).  If the number of ham words is greater than the number of spam words, assume the message is ham and vice versa.  Express the detection accuracy for ham and spam as percentages.

In [ ]:
def ham_spam(filename):
    words = get_words(filename)
    num_words = len(words)
    set_words = set(words)
    num_ham = len([True for word in set_words if word in ham_words])
    num_spam = len([True for word in set_words if word in spam_words])
    return (num_words, num_ham, num_spam)

ham_or_spam = []
for i in range(10000, 14000):
    results = ham_spam(ham_files[i])
    ham_or_spam.append(True if results[1] > results[2] else False)

ham_or_spam = np.array(ham_or_spam)
ham_accuracy = (len(ham_or_spam[np.where(ham_or_spam == True)]) / len(ham_or_spam)) * 100.0
print(f'Ham Accuracy: {ham_accuracy}')

ham_or_spam = []
for i in range(10000, 14000):
    results = ham_spam(spam_files[i])
    ham_or_spam.append(True if results[1] > results[2] else False)
ham_or_spam = np.array(ham_or_spam)
spam_accuracy = (len(ham_or_spam[np.where(ham_or_spam == False)]) / len(ham_or_spam)) * 100.0
print(f'Spam Accuracy: {spam_accuracy}')


Ham Accuracy: 86.925
Spam Accuracy: 98.125


While the spam detection accuracy looks amazing, the ham detection seems to be a bit more difficult.  Does this matter?  Is this "good enough?"

Consider this:  Would you rather have a few spam messages sneak into your inbox or a few important messages categorized as spam?  For most people, especially when we are talking about important business or personal emails, we can't afford to have them sitting in our spam folder!

Can we improve this?  Our approach so far is a very direct approach using a rudimentary statistics approach.  Let's try this again, but this time we'll use a Bayesian approach that leverages probabilities.

We discussed the basics of Bayes theorem in our lecture portion, but to implement it we need to go over a few terms.  We should also point out that *sklearn* has a Bayesian classifier built in.  We know that, but we want to walk through the implementation of a simple naive Bayes classifier to both improve our own understanding and to allow us to transport or adapt this technique to other problems.

First, a few terms.  This approach is considered *naive* not because it isn't useful but because it makes naive assumptions about the data.  The biggest assumption is that all of the terms that we will pass to the classifier are probabilistically independent.  This is a *very* useful assumption since we know that to calculate a joint probability of independent events is a simple product.  At the same time, this is naive because words are not probabilistically independent.  If an email contains the word "thank" how likely is it that the word "you" is in that email too?  We are completely (and naively) ignoring this.

Another important idea is the *prior.*  The prior, which in our case is the probability that any given message (or word) is either ham or spam.  This *could* be based on something we know.  For example, if we are iteratively processing the words in the message, then the prior is our current belief that the email is ham or spam.  What if we have no facts upon which to base our belief?  *We can choose the prior to be any arbitrary guess.*  The wonderful thing about this approach is that it is really about *updating* our belief.

The last new term we need to know is the *posterior.*  This is the value that we are trying to compute.  In our example, the posterior would be the probability that a given message is ham (or spam) given that a certain word appears in it.

# <img src="../images/task.png" width=20 height=20> Task 4.6

Making a Bayesian inference will require that we look at our data slightly differently.  For example, while we still need to know if words appear in spam or ham messages, we need to determine the *frequency* with which they appear in either.  To this end, we need to rework some of our code.

We already have a function that will return all of the words in the body of an email.  Let's leverage this.

For the first 10,000 spam messages, build a dictionary that tracks the frequency with which each word appears in spam messages.

In [ ]:
spam_word_frequency = dict()
for file in spam_files[:10000]:
    words = set(get_words(file))
    for word in words:
        if spam_word_frequency.__contains__(word)==True:
            spam_word_frequency[word] = spam_word_frequency[word]+1
        else:
            spam_word_frequency[word] = 1

ham_word_frequency = dict()
for file in ham_files[:10000]:
    words = set(get_words(file))
    for word in words:
        if ham_word_frequency.__contains__(word)==True:
            ham_word_frequency[word] = ham_word_frequency[word]+1
        else:
            ham_word_frequency[word] = 1


We now need to figure out just how hammy or spammy individual words are.  What we are finding is the probability density (the frequency of a given word in the overall distribution of words making up each class) of each word.

# <img src="../images/task.png" width=20 height=20> Task 4.7

Write a function, `spamminess()` that accepts a list of words from an email and returns a dictionary indicating the spamminess of each word in the list passed in.  Spamminess should be calculated as the number of spam emails containing the word in question divided by the total number of spam emails from our training data.

You should recognize that this will present a challenge... What if the word in question does not appear in our training data anywhere?  The simple way to avoid this problem is to add 1 to each word count so that the numerator is never zero.  Simultaneously, we should do something to the denominator, otherwise our distribution has more than 100%.  To resolve this, we should add something to the denominator as well.  Our first inclination would be to add 1 as well.  However, we will add 2 since we have two classes overall and we are making the same addition to the ham class (later).

This means that the final formula for each word would be (word_appears_in_x_emails + 1)/(total_spam_emails + 2).

In [ ]:
def spamminess(words):
    spamminess_dict = dict()
    for word in set(words):
        word_freq = spam_word_frequency[word] if word in spam_word_frequency.keys() else 0
        total_training_spam = 10000
        spamminess = (word_freq + 1) / (total_training_spam + 2)
        spamminess_dict[word] = spamminess
    return spamminess_dict


# <img src="../images/task.png" width=20 height=20> Task 4.8

We now need the same function, but for hamminess.  Please write this function.

In [ ]:
def hamminess(words):
    hamminess_dict = dict()
    for word in set(words):
        word_freq = ham_word_frequency[word] if word in ham_word_frequency.keys() else 0
        total_training_ham = 10000
        hamminess = (word_freq + 1) / (total_training_ham + 2)
        hamminess_dict[word] = hamminess
    return hamminess_dict


We're now ready to finish this up.  Remember that we started with the idea of this being naive, meaning that all of these probabilities are independent.  This allows us to compute the product of these joint probabilities:

$$S = \prod_{i=0}^n p(w_i|s)$$
$$H = \prod_{i=0}^n p(w_i|h)$$

Where $S$ is the overall spamminess and $H$ is the overall hamminess.  We can then compare those two probabilities to determine which is more likely.

# <img src="../images/task.png" width=20 height=20> Task 4.9

Combine all of the pieces we have so far to create a function that will generate the joint probabilities for hamminess and spamminess and return a three valued tuple containing a True/False for Ham/Spam, the hamminess, and the spamminess probabilities.

In [ ]:
def ham_or_spam(words):
    hamness = np.array([v for k,v in hamminess(words).items()]).prod()
    spamness = np.array([v for k,v in spamminess(words).items()]).prod()
    ham_or_spam = True if hamness > spamness else False
    return (ham_or_spam, hamness, spamness)

# <img src="../images/task.png" width=20 height=20> Task 4.10

Using ham and spam messages 10000 through 14000, calculate the accuracy of our naive Bayesian approach.

In [ ]:

ham_results = []
for i in range(10000, 14000):
    results = ham_or_spam(get_words(ham_files[i]))
    ham_results.append(results[0])

ham_results = np.array(ham_results)
ham_accuracy = (len(ham_results[np.where(ham_results==True)]) / len(ham_results)) * 100.0
print(f'Ham Accuracy: {ham_accuracy}')

spam_results = []
for i in range(10000, 14000):
    results = ham_or_spam(get_words(spam_files[i]))
    spam_results.append(results[0])

spam_results = np.array(spam_results)
spam_accuracy = (len(spam_results[np.where(spam_results==False)]) / len(spam_results)) * 100.0
print(f'Spam Accuracy: {spam_accuracy}')

Ham Accuracy: 76.4
Spam Accuracy: 99.35000000000001


Perhaps unexpectedly, our Bayesian approach produces slightly less accurate identification of ham messages!  This should give you the intuition that, even though Bayesian inferencing is used extensively, it is still not perfect.  Sometimes simpler or different models are better.

This should also give you a notion of how Bayesian filtering can be defeated.  Have you ever noticed email messages full of random words, usually at the end, that have nothing to do with the message itself?  What purpose could these serve?

# <img src="../images/task.png" width=20 height=20> Task 4.10

While the intuition explained above is useful, there is another possibility.  Use the `ham_or_spam()` function to generate raw values for each of the first five samples in our test range (10000 to 14000) for the ham and spam sets.  Can you see what might be wrong?

In [ ]:
for i in range(10000, 10005):
    print(ham_or_spam(get_words(spam_files[i])))
    print(ham_or_spam(get_words(ham_files[i])))

Notice that in several cases our product has collapsed to zero!  This means that we are running into a precision problem since we are muliplying very small values.  We could address this by using a greater precision type (perhaps 64 bit floating point).  Alternatively, we could adjust our approach to limit the number of words examined, or perhaps use an initial value that is quite large for our initial prior.  We will not attempt to solve this problem now, however, since that's not really the point of the lab.

Hopefully, understanding that this type of approach leverages the product of the joint probabilities helps you to appreciate that the people generating the junk mail are fully aware of how most spam filtering works... and how to defeat it by adding lots of words that will nudge the probability distribution toward Ham!

## Conclusion

In this lab, we had the opportunity to experiment and apply some of our statistics foundations in addition to building an understanding of the inner workings of Bayesian inference tools.  This also allowed us to learn how to defeat such approaches.

We will return to the classification of Ham and Spam in later labs as we continue to add additional tools to our arsenal.